In [1]:
import time
from Crypto.PublicKey import RSA
from Crypto.Cipher import PKCS1_OAEP
from cryptography.hazmat.primitives.asymmetric import ec
from cryptography.hazmat.primitives.kdf.hkdf import HKDF
from cryptography.hazmat.primitives.hashes import SHA256
import random

In [2]:
def generate_polynomial(secret, threshold):
    coefficients = [secret] + [random.randint(1, 1000) for _ in range(threshold - 1)]
    return coefficients

In [3]:
def evaluate_polynomial(coefficients, x):
    return sum(coef * (x ** power) for power, coef in enumerate(coefficients))

In [4]:
def reconstruct_secret(points):
    secret = 0
    for i, (x_i, y_i) in enumerate(points):
        term = y_i
        for j, (x_j, _) in enumerate(points):
            if i != j:
                term *= x_j / (x_j - x_i)
        secret += term
    print(f"Відновлений секрет: {round(secret)}")
    return round(secret)

In [5]:
def split_secret(secret, threshold, num_shares):
    coefficients = generate_polynomial(secret, threshold)
    shares = [(i, evaluate_polynomial(coefficients, i)) for i in range(1, num_shares + 1)]
    print("Частки секрету (Шамір):", shares)
    return shares

In [6]:
# Шифрування часток за допомогою RSA
def rsa_encrypt_shares(shares):
    rsa_key = RSA.generate(2048)
    public_key = rsa_key.publickey()
    cipher = PKCS1_OAEP.new(public_key)

    encrypted_shares = []
    encryption_time_start = time.time()
    for share in shares:
        encrypted_shares.append((share[0], cipher.encrypt(str(share[1]).encode())))
    encryption_time = time.time() - encryption_time_start

    print(f"Зашифровані частки (RSA): {encrypted_shares}")
    print(f"Час шифрування (RSA): {encryption_time:.5f} с")
    return rsa_key, encrypted_shares, encryption_time

In [7]:
# Розшифрування часток за допомогою RSA
def rsa_decrypt_shares(rsa_key, encrypted_shares):
    decipher = PKCS1_OAEP.new(rsa_key)

    decrypted_shares = []
    decryption_time_start = time.time()
    for share in encrypted_shares:
        decrypted_shares.append((share[0], int(decipher.decrypt(share[1]).decode())))
    decryption_time = time.time() - decryption_time_start

    print(f"Розшифровані частки (RSA): {decrypted_shares}")
    print(f"Час розшифрування (RSA): {decryption_time:.5f} с")
    return decrypted_shares, decryption_time

In [8]:
# Шифрування часток за допомогою ECC
def ecc_encrypt_shares(shares):
    private_key = ec.generate_private_key(ec.SECP256R1())
    peer_private_key = ec.generate_private_key(ec.SECP256R1())

    shared_secret = private_key.exchange(ec.ECDH(), peer_private_key.public_key())
    derived_key = HKDF(algorithm=SHA256(), length=32, salt=None, info=b'secret sharing').derive(shared_secret)

    encrypted_shares = []
    encryption_time_start = time.time()
    for share in shares:
        encrypted_shares.append((share[0], int.from_bytes(derived_key[:4], byteorder='big') ^ share[1]))
    encryption_time = time.time() - encryption_time_start

    print(f"Зашифровані частки (ECC): {encrypted_shares}")
    print(f"Час шифрування (ECC): {encryption_time:.5f} с")
    return private_key, peer_private_key, encrypted_shares, encryption_time

In [9]:
# Розшифрування часток за допомогою ECC
def ecc_decrypt_shares(private_key, peer_private_key, encrypted_shares):
    shared_secret = private_key.exchange(ec.ECDH(), peer_private_key.public_key())
    derived_key = HKDF(algorithm=SHA256(), length=32, salt=None, info=b'secret sharing').derive(shared_secret)

    decrypted_shares = []
    decryption_time_start = time.time()
    for share in encrypted_shares:
        decrypted_shares.append((share[0], int.from_bytes(derived_key[:4], byteorder='big') ^ share[1]))
    decryption_time = time.time() - decryption_time_start

    print(f"Розшифровані частки (ECC): {decrypted_shares}")
    print(f"Час розшифрування (ECC): {decryption_time:.5f} с")
    return decrypted_shares, decryption_time

In [10]:
if __name__ == "__main__":
    secret = 12345 # Початковий секрет
    threshold = 3
    num_shares = 5

    # Розділення секрету
    shares = split_secret(secret, threshold, num_shares)

    print("\n=== Розподіл часток за допомогою RSA ===")
    rsa_key, rsa_encrypted_shares, rsa_enc_time = rsa_encrypt_shares(shares)
    rsa_decrypted_shares, rsa_dec_time = rsa_decrypt_shares(rsa_key, rsa_encrypted_shares)
    rsa_reconstructed_secret = reconstruct_secret(rsa_decrypted_shares[:threshold])

    print("\n=== Розподіл часток за допомогою ECC ===")
    ecc_private_key, ecc_peer_private_key, ecc_encrypted_shares, ecc_enc_time = ecc_encrypt_shares(shares)
    ecc_decrypted_shares, ecc_dec_time = ecc_decrypt_shares(ecc_private_key, ecc_peer_private_key, ecc_encrypted_shares)
    ecc_reconstructed_secret = reconstruct_secret(ecc_decrypted_shares[:threshold])

    print("\n=== Порівняння ефективності ===")
    print(f"RSA: Час шифрування + розшифрування: {rsa_enc_time + rsa_dec_time:.5f} с")
    print(f"ECC: Час шифрування + розшифрування: {ecc_enc_time + ecc_dec_time:.5f} с")
    if rsa_enc_time + rsa_dec_time < ecc_enc_time + ecc_dec_time:
        print("RSA працює швидше для цієї задачі.")
    else:
        print("ECC працює швидше для цієї задачі.")

Частки секрету (Шамір): [(1, 12934), (2, 14085), (3, 15798), (4, 18073), (5, 20910)]

=== Розподіл часток за допомогою RSA ===
Зашифровані частки (RSA): [(1, b'\x88\xca|\r\xa9K\x0b\x9a\xab\xcf\xe5\x96\xd2\x93\x00\xbe\xe5\x89c.\xb1\xf2r\\!\xb9W\x120D\xd9\xa9\x82\xcd\xd9PQ4\x12UVq\xf6\xdd\x82\xf0\x9d\x8bbjmT>\xc6\xfa$\xd1\xb8\xf3\xd4\xa4\xb8\xad\xacK\x99H\x93\xc9_}\xdcZ\xc0c\x95\x90\x1a\x19\xb0\xb4\x87\x00\x1a8\xc82F\x87\x0b\xc40\xa2^\xc9C\x98\x04\xd1\xcf\xb9\x96~o\xe5.\x1a\xfeN1eJ\xd4\xef\xa5\x06\x03W\xc95\xb2\x02\xed\x06(\x97\xb2\x87\xefW\x92\x03\x14P)\x16\x95\x8eB\x86\x00\xd21g\x07\x80G\xa7\xbc\xf2\xf4s\x99J\xea]\xe2\xb1\xe9\x9f\xfa\xe2H\xbdy\x90\x96\xa8|\xa3\xdc\x15\xe83\x83_S\xd1v\xcd\xc6\x9a\x010\xe1\xc2\x07M\x80T\xc3\x1f1:\x7fj\x0c\xa7\xba\xc8\xa4\xc6]r\x15\xac;\x1e\x8d\x8a\xdbp\x89G|5\x1c\xd2 \x1e[\xe8\x06\xd8\x17\xc9k\xad\x80\xed\x14\xef\xdd\xc9\xa3\xe5\x02\xdd:~\x87\xba\x1d{[\x00\xf1\x99ek\xa6\x12\xdb(7\xe6'), (2, b'Bz\x1d}\\\xb5\x13\xder\xbdj&ZM\x7f\x12\x89%\xc7\xfb\x04;\xc1\x